In [ ]:
import numpy as np
import scipy.io.wavfile as wav
import csv
import os
from tqdm import tqdm
import torch
import torchaudio
import torchaudio.transforms as T
import argparse

parser = argparse.ArgumentParser(description='Add noise to audio files')
parser.add_argument('--data_split', type=str, default="/old_scratch/bbjs/ylu3/SpeechCLIP/data/flickr/flickr_dev_sampled.csv")
parser.add_argument("--data_root", type=str, default="/old_scratch/bbjs/ylu3/SpeechCLIP/data/flickr/flickr_audio/wavs")
parser.add_argument('--output_dir', type=str, default="/old_scratch/bbjs/ylu3/tmp/flickr/noisy_dev_sampled")
parser.add_argument('--snr', type=int, default=20)

args = parser.parse_args()

data_file = args.data_split
output_dir = f"{args.output_dir}_{args.snr}"
os.makedirs(output_dir, exist_ok=True)
data = []
with open(data_file, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

for i, d in tqdm(enumerate(data[1:])):
    input_path = f"{args.data_root}/{d[2]}"
    if os.path.exists(input_path):
        waveform, sample_rate = torchaudio.load(input_path)
        noise = torch.randn_like(waveform)
        add_noise = T.AddNoise()
        noisy_waveform = add_noise(waveform, noise, snr=torch.tensor([args.snr]))
        torchaudio.save(f"{output_dir}/{d[2]}", noisy_waveform, sample_rate)
    else:
        print(f"{input_path} does not exist")

